# Explore Data Lake

## Utils

In [0]:
DATAHUB_PATH = "dbfs:/mnt/datahub"
DL_PATH_BRONZE = f"{DATAHUB_PATH}/bronze"
DL_PATH_SILVER = f"{DATAHUB_PATH}/silver"
DL_PATH_DEMO = f"{DATAHUB_PATH}/demo"

DATABASE = "customersales"
SCHEMA = "dbo"

In [0]:
def get_bronze_table_path(schema: str, tablename: str) -> str:
    return f"{DL_PATH_BRONZE}/{DATABASE}/{schema}/{tablename}.parquet"


def get_write_table_path(container_path: str, schema: str, tablename: str) -> str:
    return f"{container_path}/{DATABASE}/{schema}/{tablename}"


print(get_bronze_table_path(SCHEMA, "distributors"))
print(get_write_table_path(DL_PATH_SILVER, SCHEMA, "distributors"))


In [0]:
df = spark.read.parquet(get_bronze_table_path(SCHEMA, "distributors"))
display(df)

In [0]:
tablename = get_write_table_path(DL_PATH_DEMO, SCHEMA, "distributors")
print(tablename)

In [0]:
spark.sql(f"""
    CREATE DATABASE IF NOT EXISTS demolakehouse
    LOCATION '/mnt/datahub/demo'
""")

In [0]:
%sql
SHOW DATABASES

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("demolakehouse.dbo_distributors")

In [0]:
%run "/Workspace/Users/rico.goerlitz@gmail.com/explore-azure-databricks-project/datahub/databricks/git/100 etl/001 MASTER NOTEBOOK/MASTER_SILVERINGESTION"

In [0]:
print(etl)

In [0]:
etl.init(
    spark=spark,
    src_lakehouse="customersales",
    src_schema="dbo",
    src_tablename="regions",
    dist_lakehouse="demolakehouse",
    dist_schema="dbo",
    dist_tablename="regions",
    nk_columns=["id"],
    constant_columns=None,
    is_delta_load=False,
    delta_load_use_broadcast=False,
    transformations=None,
    exclude_comparing_columns=[],
    include_comparing_columns=[],
    historize=True,
    partition_by_columns=[],
)

etl.ingest()